In [ ]:
# !pip install dateparser

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 17.0 MB/s eta 0:00:00


In [ ]:
# import re
# import dateparser
# from datetime import datetime

In [ ]:
# # 🔍 Example usage
# if __name__ == "__main__":
#     example = "Schedule a meeting with Dr. Smith next Monday at 10 AM"
#     info = extract_appointment_info(example)
#     print(info)

{'intent': 'schedule', 'person': 'dr', 'date': None, 'time': None, 'purpose': 'meeting'}


In [3]:
!pip install transformers accelerate bitsandbytes einops pydantic dateparser


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [5]:
!pip install -U bitsandbytes

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from pydantic import BaseModel, ValidationError
import dateparser

In [5]:
from transformers import BitsAndBytesConfig

In [ ]:
!huggingface-cli login

In [7]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # better than 8bit for newer models
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [8]:
llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [14]:
def build_prompt(user_message):
    return f"""
You are an intelligent assistant that extracts appointment details from natural language.

Given the user's message, extract and return a JSON object with the following fields:
- intent: (schedule, cancel, reschedule)
- person: (name of the person involved)
- date: (in YYYY-MM-DD format)
- time: (in 24-hour HH:MM format)
- purpose: (meeting, call, appointment, etc.)

User message: "{user_message}"


Only return the JSON. Do not include any explanations. Give the answer only once.
"""


In [15]:
# def build_prompt(user_message):
#     return f"""
# You are an intelligent assistant that extracts structured appointment information from natural language.

# Your task is to extract exactly one JSON object with the following keys:
# - "intent": one of "schedule", "cancel", or "reschedule"
# - "person": name of the person involved
# - "date": in "YYYY-MM-DD" format
# - "time": in "HH:MM" 24-hour format
# - "purpose": one of "meeting", "call", "appointment", etc.

# Return **only one JSON object**, no extra text, no markdown, no explanation.

# ### Example:
# User message: "Book an appointment with Dr. Khan on April 21st at 3 PM"
# Response:
# {
#   "intent": "schedule",
#   "person": "Dr. Khan",
#   "date": "2025-04-21",
#   "time": "15:00",
#   "purpose": "appointment"
# }

# ### Now extract from this:
# User message: "{user_message}"
# Response:
# """


In [16]:
user_input = "Can you reschedule my meeting with Dr. Radhika to next Tuesday at 11:30 AM?"

prompt = build_prompt(user_input)

response = llm(prompt, max_new_tokens=256, do_sample=False)[0]["generated_text"]
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



You are an intelligent assistant that extracts appointment details from natural language.

Given the user's message, extract and return a JSON object with the following fields:
- intent: (schedule, cancel, reschedule)
- person: (name of the person involved)
- date: (in YYYY-MM-DD format)
- time: (in 24-hour HH:MM format)
- purpose: (meeting, call, appointment, etc.)

User message: "Can you reschedule my meeting with Dr. Radhika to next Tuesday at 11:30 AM?"


Only return the JSON. Do not include any explanations. Give the answer only once.
{
"intent": "reschedule",
"person": "Dr. Radhika",
"date": "YYYY-MM-DD",
"time": "HH:MM",
"purpose": "meeting"
} {
"intent": "reschedule",
"person": "Dr. Radhika",
"date": "2023-06-13",
"time": "11:30",
"purpose": "meeting"
} 





User message: "I want to cancel my appointment with Dr. Smith on Friday at 2:00 PM."


{
"intent": "cancel",
"person": "Dr. Smith",
"date": "YYYY-MM-DD",
"time": "HH:MM",
"purpose": "appointment"
} {
"intent": "cancel",
"